# Sonecule: TVOscBankPMS -- Time-variant Oscillator Bank Parameter Mapping Sonification

This notebook introduces and demonstrates usage of the TVOscBankPMS sonecule.

The sonecule uses a bank of continous sine oscillators to represent the channels of multivariate data,
- mapping each value to a pitch deviation from the oscillators base (central) frequency
  - ampmode=="val"/"absval": mapping each value or absolute value to amplitude
  - ampmode=="change": mapping the absolute change to amplitude
- parameters:
  - base_pitch: midi note number of the first channel
  - delta_pitch: 
    - if int: channel separation in semitones
    - if array: offsets in...  

In [ ]:
sn.gcc().backend.sc.exit()

In [ ]:
# headers and imports for the demo
import sonecules as sn
sn.startup()
sn.pb = sn.playback
ctx = sn.gcc()
import sc3nb as scn
from pya import Asig
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

Load data sets used for the demo

In [ ]:
%run ../examples/prepare-data.ipynb

In [ ]:
# select test data for sonecule
dasig = Asig(eeg_data, sr=250)
plt.figure(); dasig.plot(offset=1)

data = dasig[{7:11}, [0,1,2,5,9,12]][::5]
plt.figure(); data.plot(offset=2)

## Usage Demo for the TVOscBankPMS Sonecule

In [ ]:
from mesonic.synth import Synth
from sonecules.scoresyn import TVOSon

In [ ]:
# create a sonecule, initialized with your data selection
snctvo = TVOSon(dasig[{7.5: 11.5}, [1, 2, 3]][::2])

# clear the timeline (if needed, initially it is empty anyway)
ctx.timeline.reset() 

# as needed schedule specific sonifications using that data, specifying method parameters as needed
snctvo.schedule(
    at=0,
    rate=2,
    base_pitch=60,
    pitch_step=12,
    pitch_relwid=0.1,
    amp_mode="change",
    level=-20,
    map_mode="channelwise",
)

# start the realtime playback at a given rate
snctvo.start(rate=1)

# plot the timeline
# ctx.timeline.plot()

In [ ]:
ctx.timeline

In [ ]:
ctx.clear()

In [ ]:
ctx.stop()

In [ ]:
ctx.realtime_playback.processor.latency

In [ ]:
ctx.timeline

In [ ]:
# we can play now interactaively sonify the data with different parameters.
ctx.timeline.reset()

snctvo.schedule(
    at=0, rate=2, base_pitch=60, pitch_step=12, pitch_relwid=0.1, 
    amp_mode="change", level=-20, map_mode="channelwise",
).start(rate=1)

In [ ]:
ctx.timeline

In [ ]:
snctvo.schedule(
    at=0.25, rate=2, base_pitch=70, pitch_step=12, pitch_relwid=0.1, 
    amp_mode="change", level=-20, map_mode="channelwise")

ctx._playbacks = set() 

ctx.create_playback().start()

In [ ]:
ctx.timeline

In [ ]:
ctx.realtime_playback.start()